# Map Match QAQC

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from leuvenmapmatching.matcher.distance import DistanceMatcher
from leuvenmapmatching.map.inmem import InMemMap
from leuvenmapmatching import visualization as mmviz
import pickle
import time
import datetime
from pathlib import Path
from tqdm import tqdm
from shapely.ops import Point, LineString
import matplotlib.pyplot as plt

from importlib import reload

import map_match

In [ ]:
import json
config = json.load((Path.cwd().parent / 'config.json').open('rb'))
export_fp = Path(config['project_directory']) / 'Calibration'
cycleatl_fp = Path(config['project_directory']) / 'CycleAtlanta'
matching_fp = Path(config['project_directory']) / 'Map_Matching'
network_fp = Path(config['project_directory']) / 'Network'
if export_fp.exists() == False:
    export_fp.mkdir()

In [ ]:
matching_index = 0

# export 
with (matching_fp/f'matched_0.pkl').open('rb') as fh:
    match_dict = pickle.load(fh)

# # export the matching settings tested
# with (export_fp/'matching_settings_df.pkl').open('wb') as fh:
#     pickle.dump(matching_settings_df,fh)

In [ ]:
#for exmamining trips 
if (matching_fp/'qaqc.pkl').exists():
    with (matching_fp/'qaqc.pkl').open('rb') as fh:
        qaqc_dict = pickle.load(fh)
else:
    qaqc_dict = {}

In [ ]:
qaqc_dict

In [ ]:
with (network_fp.parent / 'chosen.pkl').open('rb') as fh:
    df_edges,pseudo_df,pseudo_G = pickle.load(fh)

In [ ]:
#add the geometry 
fp = Path.home() / "Documents/BikewaySimData/Projects/gdot"
edges = gpd.read_file(fp/'networks/elevation_added.gpkg',layer="links")
edges.to_crs('epsg:2240',inplace=True)
df_edges = df_edges.merge(edges[['linkid','geometry']],on=['linkid'])
df_edges = gpd.GeoDataFrame(df_edges,geometry='geometry',crs=edges.crs)
df_edges = df_edges.loc[:,~df_edges.columns.duplicated()].copy()
df_edges.reset_index(drop=True,inplace=True)

In [ ]:
import time
from IPython.display import display, clear_output

from importlib import reload
reload(map_match)

for tripid in match_dict.keys():

    if isinstance(match_dict.get(tripid,0),str):
        qaqc_dict[(tripid,matching_index)] = 'failed match'
        continue


#TODO get condition that won't write to dict if a trip is skipped

    if qaqc_dict.get((tripid,matching_index),0) == 0:

        html_map = map_match.visualize_match(tripid, match_dict, df_edges)
        clear_output(wait=True)
        display(html_map)
        
        # Wait for user input to proceed to the next trip
        user_input = input("Rate from 1-5 with 1 being no match to 5 being perfect match (press 'enter' to skip or 'q' to quit)")
        
        if user_input.lower() == 'q':
            break  # Exit the loop if the user enters 'q'
        if user_input == '':
            continue

        notes = input("Input notes if desired and press enter")

        #save user input for that matching index
        qaqc_dict[(tripid,matching_index)] = {
            'rating': user_input,
            'notes': notes,
            'last_reviewed': datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    #prev_qaqc = qaqc_dict.get((tripid,matching_index))
    #html_map = map_match.visualize_match(tripid, match_dict, edges, {'prev_qaqc':prev_qaqc})

#TODO add the trip date to viz
        

# Additional cleanup or actions after examining all trips can be added here
print("Finished examining trips.")

In [ ]:
550

In [ ]:
match_dict[tripid]['trace']

In [ ]:
df_edges.columns

In [ ]:
with (export_fp/'qaqc.pkl').open('wb') as fh:
    pickle.dump(qaqc_dict,fh)